In [16]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from helpers import *
import time

# '''
# Calculate the Black-Scholes implied volatility.

# Parameters:	
# price (float) – the Black-Scholes option price
# S (float) – underlying asset price
# K (float) – strike price
# t (float) – time to expiration in years
# r (float) – risk-free interest rate
# flag (str) – ‘c’ or ‘p’ for call or put.
# >>> S = 100
# >>> K = 100
# >>> sigma = .2
# >>> r = .01
# >>> flag = 'c'
# >>> t = .5
# >>> price = black_scholes(flag, S, K, t, r, sigma)
# >>> iv = implied_volatility(price, S, K, t, r, flag)
# '''

# Alpha Vantage API Key
# 5HZEUI5AFJB06BUK

# ts = TimeSeries(key='5HZEUI5AFJB06BUK', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# For intraday
# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=d5HZEUI5AFJB06BUK&datatype=csv

# For daily
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=5HZEUI5AFJB06BUK&datatype=csv

# Allocate Smartly Checking

In [ ]:
portfolio_value = 50000

as_lst = ['DBC','IEF','IWM','QQQ','SPY','VGK','VNQ']
as_wgt = [0.028, 0.333, 0.028, 0.028, 0.528, 0.028, 0.028]

as_df = pd.DataFrame({'Allocation': as_wgt}, index = as_lst)
curr_spot = curr_batch_quotes(as_lst)
as_df = pd.concat([as_df,curr_spot[['price']]], axis = 1)

as_df['Market Value'] = portfolio_value*as_df.Allocation
as_df['Holdings'] = np.round(as_df['Market Value']/as_df.price,0)

as_df

# Earnings Checking

In [ ]:
start_time = time.time()

earnings_lst = []
starting_date = dt.datetime(2018,8,2)


for date in [starting_date + dt.timedelta(i) for i in range(2)]:
    curr_earning = yahoo_earnings(date)
    curr_earning['Earnings Date'] = date
    earnings_lst.append(curr_earning)
earnings = pd.concat(earnings_lst,axis = 0)
tick_lst = earnings.Tickers.tolist()
vols = current_volatility(tick_lst, roll = 22)
earnings.index = earnings.Tickers
earnings_df = earnings.drop_duplicates().join(vols, how='right').drop_duplicates()
earnings_df['Lower Bound'] = earnings_df['close']*(1 - earnings_df['Implied Move Weekly']/100)
earnings_df['Upper Bound'] = earnings_df['close']*(1 + earnings_df['Implied Move Weekly']/100)
print("--- %s seconds ---" % (time.time() - start_time))
earnings_df.index = earnings_df.Tickers
#earnings_df.sort_values('Earnings Date')

#Display Portion
names = earnings_df[((earnings_df['Call Times'] == 'After Market Close') & 
                     (earnings_df['Earnings Date'] == starting_date)) |
                    ((earnings_df['Call Times'] == 'Before Market Open') & 
                     (earnings_df['Earnings Date'] == starting_date + dt.timedelta(1)))].sort_values('Earnings Date')

names[names['close'] >= 80]

In [ ]:
start_time = time.time()

focus = ['EOG','CBOE']
focus_vol = current_volatility(focus, roll = 22)
focus_names = focus_vol.join(names[['Call Times', 'Implied Move Weekly', 'Earnings Date']])

focus_names['Lower Bound'] = focus_names['close']*(1 - focus_names['Implied Move Weekly']/100)
focus_names['Upper Bound'] = focus_names['close']*(1 + focus_names['Implied Move Weekly']/100)

print("--- %s seconds ---" % (time.time() - start_time))

focus_names

In [ ]:
start_time = time.time()

max_gap = 5
dte_thresh = 5
money_thresh = 0.1
delta_thresh = 0.03
minimum_prem = 0.15
condors_lst = []
putspread_lst = []
callspread_lst = []
bounds_adj = 0.00

for tick in focus_names.index.drop_duplicates().tolist():
    condors, put_spreads, call_spreads = earnings_condor(tick, max_gap, dte_thresh, money_thresh)
    condors['Ticker'] = tick
    put_spreads['Ticker'] = tick
    call_spreads['Ticker'] = tick
    condors = condors[(abs(condors['Delta']) <= delta_thresh) &
                (condors['P Short Strike'] <= focus_names.loc[tick]['Lower Bound']*(1 - bounds_adj)) &
                (condors['C Short Strike'] >= focus_names.loc[tick]['Upper Bound']*(1 + bounds_adj)) &
                (condors['Premium'] >= minimum_prem)].sort_values('RiskRewardRatio', ascending = False)
    
    put_spreads = put_spreads[(put_spreads['Short Put Strike'] <= focus_names.loc[tick]['Lower Bound']) &
                              (put_spreads['put Spread Premium'] >= minimum_prem)].sort_values('RiskRewardRatio', ascending = False)
    
    call_spreads = call_spreads[(call_spreads['Short call Strike'] >= focus_names.loc[tick]['Upper Bound']) &
                                (call_spreads['call Spread Premium'] >= minimum_prem)].sort_values('RiskRewardRatio', ascending = False)
    
    condors_lst.append(condors)
    putspread_lst.append(put_spreads)
    callspread_lst.append(call_spreads)

earnings_pspreads = pd.concat(putspread_lst).sort_values('RiskRewardRatio', ascending = False)
earnings_cspreads = pd.concat(callspread_lst).sort_values('RiskRewardRatio', ascending = False)
earnings_condors = pd.concat(condors_lst).sort_values('RiskRewardRatio', ascending = False)

print("--- %s seconds ---" % (time.time() - start_time))

prem_thresh = 0.3
rr_thresh = 0.3

filtered_condors = earnings_condors[(earnings_condors['Premium'] >= prem_thresh) &
                                     (earnings_condors['RiskRewardRatio'] >= rr_thresh)]

filtered_pspreads = earnings_pspreads[(earnings_pspreads['put Spread Premium'] >= prem_thresh) &
                                     (earnings_pspreads['RiskRewardRatio'] >= rr_thresh)]

filtered_cspreads = earnings_cspreads[(earnings_cspreads['call Spread Premium'] >= prem_thresh) &
                                     (earnings_cspreads['RiskRewardRatio'] >= rr_thresh)]

filtered_condors

# SPX Options Checking

In [ ]:
start_time = time.time()

dte_thresh = 2
money_thresh = 0.02
tick = '^SPX'
deltas_thresh = [-0.5,-0.01]

spx_options = all_options(tick)

spx_filtered_puts = spx_options[(spx_options['DTE'] <= dte_thresh) &
                                (spx_options['Type'] == 'put') &
                                (spx_options['Delta'] >= deltas_thresh[0]) &
                                (spx_options['Delta'] <= deltas_thresh[1])].reset_index()[spx_options.columns].sort_values('Strike',
                                                                                                                ascending = False)

spreads_lst = []
for idx, row in spx_filtered_puts.iterrows():
    if idx == 0:
        break
    
    temp_chains = spx_filtered_puts[spx_filtered_puts['Strike'] < row.Strike][['Strike','Bid','Ask',
                                                                               'Delta','Gamma','Theta','Vega']]
    temp_chains['DTE'] = row.DTE
    temp_chains['Short Strike'] = row.Strike
    temp_chains['Short Bid'] = row.Bid
    temp_chains['Short Ask'] = row.Ask
    
    temp_chains['Premium'] = (temp_chains['Short Bid'] - temp_chains['Ask'])*100
    temp_chains['Total Risk'] = (temp_chains['Strike'] - temp_chains['Short Strike'])*100 + temp_chains['Premium']
    temp_chains['Delta'] = temp_chains['Delta'] - row.Delta
    temp_chains['Gamma'] = temp_chains['Gamma'] - row.Gamma
    temp_chains['Theta'] = temp_chains['Theta'] - row.Theta
    temp_chains['Vega'] = temp_chains['Vega'] - row.Vega
    
    temp_chains = temp_chains[['DTE','Short Strike','Strike','Premium','Delta','Gamma','Theta','Vega','Total Risk']]
    spreads_lst.append(temp_chains)
    
pspreads_df = pd.concat(spreads_lst, axis = 0).reset_index()[temp_chains.columns]
pspreads_df['RRRatio'] = pspreads_df.Premium/(-pspreads_df['Total Risk'])

pspreads_df = pspreads_df.sort_values('RRRatio',ascending = False)

print("--- %s seconds ---" % (time.time() - start_time))